# МОДУЛИ

In [1]:
import os
import time
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

from skimage.io import imread  # Чтение файла сразу в np массив, для варианта с обрезкой

import tensorflow as tf
from tensorflow.keras.preprocessing import image  # Для работы с изображениями 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB7
from keras.applications.imagenet_utils import decode_predictions  # Для декодирования лейблов


In [3]:
start_time = time.time()

In [2]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

Mounted at /content/drive


# ЗАГРУЗКА МОДЕЛИ

In [4]:
model = EfficientNetB7(weights='imagenet')

268337152/268326632 [==============================] - 3s 0us/step


# НАСТРОЙКИ

In [5]:
TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/files/test'  # Тестовые данные
MODEL_DIR = '/content/drive/MyDrive/Colab Notebooks/files/model'  # Модель
IMG_SIZE = 600  # Размер изображения для нейросети
BATCH_SIZE = 1
THRESHOLD = 0.005

# ФУНКЦИИ

## Функция вывод изображений

In [6]:
# Функция вывода изображений
def plot_image(img_np, title_list, row=4, col=5, size=5):
  ''' Настройки вывода изображения
  row = 4  # Строк
  col = 5  # Колонок
  size = 5  # Размер изображения, в дюймах
  img_np - массив numpy
  title_list - список заголовков
  '''

  fig, axs = plt.subplots(row, col, figsize=(col*size, row*size))  # Создаем полотно

  for r in range(row):
    for c in range(col):
      num = r*col + c
      axs[r, c].axis('off')
      img = img_np[num]/255
      axs[r, c].imshow(img)  # Отображаем фото
      title = title_list[num] if len(title_list[num]) < 20 else title_list[num][:20] + '...'
      axs[r, c].set_title(label=title, y=-0.1)  # Выводим заголовок, с у позицией

## Predict

In [7]:
def get_predict(file_name):
  path = os.path.join(TEST_DIR , file_name)
  data_list = []
  img = image.load_img(
    path = path, 
    color_mode='rgb', 
    target_size=(IMG_SIZE, IMG_SIZE),
  )
  np_img = image.img_to_array(img)
  data_list.append(np_img)

  x_data = np.array(data_list)

  y = model.predict(x_data)
  predict = decode_predictions(y, top=10)

  for pred in predict:  # Проходим по изображениям
    class_num = 3  # По умолчанию - ничего не найдено
    for p in pred:  # Проходим по предсказаниям изображения (по умолчанию 5)
      if p[1] == 'tiger' and p[2] > THRESHOLD:
        class_num = 1
        break 
      if p[1] == 'leopard' and p[2] > THRESHOLD:
        class_num = 2
        break

  return class_num


# ДАННЫЕ

In [8]:
files_list = os.listdir(TEST_DIR)
images_list = list(filter(lambda x: x.endswith('.jpg'), files_list))
print('Всего изображений:', len(images_list))


Всего изображений: 546


In [9]:
predict_list = []
i = 0
for file_name in images_list:
  class_num = get_predict(file_name)
  predict_list.append([file_name, class_num])
  if (i) % 10 == 0:
    print(f'{i}, {round(time.time() - start_time)}c')
  i += 1

print(predict_list)

49152/35363 [=========================================] - 0s 0us/step
0, 98c
10, 103c
20, 109c
30, 114c
40, 119c
50, 125c
60, 130c
70, 135c
80, 140c
90, 146c
100, 151c
110, 156c
120, 161c
130, 166c
140, 171c
150, 176c
160, 181c
170, 186c
180, 192c
190, 197c
200, 202c
210, 208c
220, 213c
230, 218c
240, 224c
250, 229c
260, 234c
270, 240c
280, 245c
290, 250c
300, 256c
310, 261c
320, 266c
330, 271c
340, 275c
350, 281c
360, 286c
370, 291c
380, 296c
390, 301c
400, 306c
410, 311c
420, 317c
430, 322c
440, 327c
450, 332c
460, 338c
470, 344c
480, 349c
490, 356c
500, 363c
510, 369c
520, 376c
530, 383c
540, 389c
[['001_95.jpg', 1], ['8_IMAG0123_S02.jpg', 2], ['001_93.jpg', 1], ['001_90.jpg', 3], ['001_92.jpg', 1], ['001_9.jpg', 1], ['7_IMAG1444_S07.jpg', 3], ['001_87.jpg', 1], ['001_89.jpg', 3], ['001_86.jpg', 1], ['001_84.jpg', 3], ['001_764.jpg', 3], ['7_IMAG1263_S10.jpg', 3], ['10_IMAG0022_S02.jpg', 1], ['109_IMG_0369_S02.jpg', 3], ['10_IMAG0125_S02.jpg', 2], ['109_IMG_0367_S02.jpg', 3], ['109_

In [ ]:
df = pd.DataFrame(predict_list)
print(df.shape)
df.head()

(1000, 2)


,0,1
0,001_102.jpg,3
1,001_103.jpg,3
2,001_104.jpg,3
3,001_106.jpg,1
4,001_108.jpg,1


In [ ]:
df.to_csv(TEST_DIR+'/RESULT_3.csv', sep=',', encoding='utf-8', index=False, header=False)

In [ ]:
print(f'Время выполнения: {round(time.time() - start_time)}c')

Время выполнения: 580c
